In [2]:
import logging
import warnings

from dotenv import load_dotenv

from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)
load_dotenv()


True

In [3]:
CHOSEN_MODEL = "gemini-1.5-flash"

# Setting up the agents

In [4]:
import tools

GREETING_AGENT_INSTRUCTIONS = """
You are the Greeting Agent. Your ONLY taks is to provide a firently greeting to the user.
Use 'say_hello' tool to generate a greeting.
If the user provides their name, make sure to pass it to the tool.

Do not engate in any other conversation or tasks.
"""

greeting_agent = Agent(
    name="greeting_agent_v1",
    description="Handles simple greetings using the 'say_goodbye' tool",
    instruction=GREETING_AGENT_INSTRUCTIONS,
    tools=[tools.say_hello],
    model=CHOSEN_MODEL,
)

GOODBYE_AGENT_INSTRUCTIONS = """
You are the Farewell Agent. Your ONLY task is to provide a friendly farewell to the user.
Use 'say_goodbye' tool when the user indicates they are leaving or ending the conversation.
(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you').
If the user provides their name, make sure to pass it to the tool.
Do not perform any other actions or tasks.
"""

goodbye_agent = Agent(
    name="goodbye_agent_v1",
    description="Handles simple farewells using the 'say_goodbye' tool",
    instruction=GOODBYE_AGENT_INSTRUCTIONS,
    tools=[tools.say_goodbye],
    model=CHOSEN_MODEL,
)

# Setting up the ROOT agent

In [5]:
WEATHER_AGENT_INSTRUCTIONS = """
You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. 
Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London').
You have specialized sub-agents: "
1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these.
2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these.
Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'.
If it's a weather request, handle it yourself using 'get_weather'.
For anything else, respond appropriately or state you cannot handle it.
"""

weather_agent_team = Agent(
    name="weather_agent_v2",
    model=CHOSEN_MODEL,
    description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
    instruction=WEATHER_AGENT_INSTRUCTIONS,
    sub_agents=[greeting_agent, goodbye_agent],
    tools=[tools.get_weather],
)

In [6]:
APP_NAME = "weather_tutorial_app"
USER_ID = "user_123"
SESSION_ID = "session_123"

session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    agent=weather_agent_team,
    app_name=APP_NAME,
    session_service=session_service,
)

runner_args = {"runner": runner, "user_id": USER_ID, "session_id": SESSION_ID}

In [7]:
from agent_call_utils import call_agent_async

await call_agent_async(query="Hello there, I'm Ilya!", **runner_args)
await call_agent_async(query="What is the weather in Paris?", **runner_args)
await call_agent_async(query="Thanks, bye! Call me by my name", **runner_args)



{
    "content": {
        "parts": [
            {
                "video_metadata": null,
                "thought": null,
                "inline_data": null,
                "file_data": null,
                "thought_signature": null,
                "code_execution_result": null,
                "executable_code": null,
                "function_call": {
                    "id": "adk-eea482cb-9d5b-48dd-8404-ac389e4ed9fd",
                    "args": {
                        "agent_name": "greeting_agent_v1"
                    },
                    "name": "transfer_to_agent"
                },
                "function_response": null,
                "text": null
            }
        ],
        "role": "model"
    },
    "grounding_metadata": null,
    "partial": null,
    "turn_complete": null,
    "error_code": null,
    "error_message": null,
    "interrupted": null,
    "custom_metadata": null,
    "usage_metadata": {
        "cache_tokens_details": null,
        "cache